# Audio model evaluation

In [2]:
import os
import pickle
import pandas as pd
from pyAudioAnalysis import audioTrainTest as aT

labels = pd.read_csv("project_data/labels.csv", usecols=[0,1], dtype={"id": int, "label": str}, index_col=0)

def get_label_by_story(subject, story):
    return labels.loc[subject].values[0][story-1]

def lou_cross_validation_audio(audio_path, save_results=None):
    save_results = audio_path if save_results==None else save_results
    all_files = []
    for root, dirs, files in os.walk(audio_path):
        for f in files:
            all_files.append(os.path.join(root, f))
    predictions = {}
    for trfile in all_files:
        originalname = trfile
        inter = originalname.split("/")[-1].strip("sub").strip(".wav").split("_")
        sub, st = int(inter[0]), int(inter[1])
        movedname = originalname.replace(originalname.split("/")[1], "fold")
        os.rename(originalname, movedname)
        aT.featureAndTrain([audio_path + "/lie", audio_path + "/truth"], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "svm", "audio_models/svm/svm", True)
        prediction = aT.fileClassification(movedname, "audio_models/svm/svm", "svm")[1][1]
        prediction = 1 if prediction>0.5 else 0
        predictions[(sub, st)] = [prediction, get_label_by_story(sub, st)]
        os.rename(movedname, originalname)

    with open(save_results + ".pickle", "wb") as f:
        pickle.dump(predictions, f)

control_group = "audio_control_group"
test_group = "audio_test_group"
lou_cross_validation_audio(control_group)
lou_cross_validation_audio(test_group)

In [40]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse

def deserialize(f):
    with open(f, "rb") as ser:
        return pickle.load(ser)


def evaluate_predictions(filein):
    audio_prediction = deserialize(filein)
    results = np.array(audio_prediction.values()).astype(int)
#     results mse(results[:, 0], results[:, 1])
    results = results[:, 0] - results[:, 1]
    results = (len(audio_prediction) - sum(abs(results)))/float(len(audio_prediction))
    print "Correct classification rate of {}: {:.4f}%".format(filein.strip(".pickle"), results*100)

# evaluate_predictions("audio_control_group.pickle")
# evaluate_predictions("audio_test_group.pickle")
evaluate_predictions("audio_complete_group.pickle")


# Thermal model evaluation